# IMDB Movies

Goal: Create a MySQL database that catalogues movies, and analyze what makes a movie successful or not.

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd

In [14]:
#Setting pandas options to max column and row displays
pd.set_option('display.max_columns', None) #Used for displaying columns
pd.set_option('display.max_rows', None) #Used for displaying rows

## Part 1: Creating and Cleaning Pandas Databases

* replace all "\N" values with np.nan
* remove all movies that are not localized in the US
* For Basics:
    * remove movies without a genre
    * only keep movies
    * only use movies from 2000-2022 (inclusive)
    

In [2]:
# Creating url paths from original links
# DEFUNCT: data has been cleaned and is available in data folder
#basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
#ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
#akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [57]:
#Reading url paths into databases
# DEFUNCT: data has been cleaned and is available in data folder
#basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
#ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
#akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [12]:
#Determining where to search for US titles
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [15]:
#Checking what abbreviation = US
#i.e. is the abbr. US, or USA?
akas["region"].value_counts()

DE      4226415
FR      4222606
JP      4221949
IN      4161606
ES      4143884
IT      4124087
PT      4055877
\N      1884624
US      1424064
GB       444404
CA       221867
XWW      171062
AU       131056
BR       116765
RU        94360
MX        93548
GR        91908
PL        87633
FI        86673
SE        76011
HU        74038
NL        62578
AR        59445
NO        55872
PH        55837
DK        54841
TR        52312
XWG       51511
SUHH      37409
HK        34027
BE        33023
KR        32146
ZA        30569
TW        30439
CN        29650
AT        29582
RO        28230
BG        27699
SG        27645
UA        24511
RS        22587
CZ        20188
IL        19817
ID        18779
AE        16320
IE        16149
XYU       15599
EG        14803
HR        13977
VE        13123
CH        13015
IR        12817
NZ        12725
TH        11829
LT        11522
VN        11206
CL        10203
CSHH       9802
DDDE       9643
SI         9408
CO         8829
NG         8522
SK      

In [35]:
#Cleaning akas
is_in_US = akas["region"] == "US"
akas = akas[is_in_US]
akas = akas.replace({"\\N":np.nan})

akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [34]:
#Cleaning ratings
US_filter = ratings["tconst"].isin(akas["titleId"])
ratings = ratings[US_filter]

ratings = ratings.replace({"\\N": np.nan})

ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1961
1,tt0000002,5.8,263
4,tt0000005,6.2,2600
5,tt0000006,5.1,178
6,tt0000007,5.4,817


In [52]:
#Cleaning basics
US_filter = basics["tconst"].isin(akas["titleId"])
basics = basics[US_filter]
basics = basics.replace({"\\N": np.nan})

#Filtering for only movies
isMovie = basics["titleType"] == "movie"
basics = basics[isMovie]

#Removing documentaries
isDocumentary = basics["genres"].str.contains("documentary", na = False)
basics = basics[~isDocumentary]

#Removing movies before 2000
isOlderThan2000 = basics["startYear"] >= "2000"
basics = basics[isOlderThan2000]

#Removing movies after 2022
isYoungerThan2022 = basics["startYear"] <= "2022"
basics = basics[isYoungerThan2022]

#removing null runtimeMinutes
isNullMinutes = basics["runtimeMinutes"] > ""
basics = basics[isNullMinutes]


isNullGenre = basics["genres"] > ""
basics = basics[isNullGenre]



basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [56]:
#Committing files to data folder
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [53]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119911 entries, 34803 to 9710750
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          119911 non-null  object 
 1   titleType       119911 non-null  object 
 2   primaryTitle    119911 non-null  object 
 3   originalTitle   119911 non-null  object 
 4   isAdult         119911 non-null  object 
 5   startYear       119911 non-null  object 
 6   endYear         0 non-null       float64
 7   runtimeMinutes  119911 non-null  object 
 8   genres          119911 non-null  object 
dtypes: float64(1), object(8)
memory usage: 9.1+ MB


In [54]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424064 entries, 5 to 35330135
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1424064 non-null  object
 1   ordering         1424064 non-null  int64 
 2   title            1424064 non-null  object
 3   region           1424064 non-null  object
 4   language         3866 non-null     object
 5   types            976227 non-null   object
 6   attributes       46241 non-null    object
 7   isOriginalTitle  1422719 non-null  object
dtypes: int64(1), object(7)
memory usage: 97.8+ MB


In [55]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492984 entries, 0 to 1292576
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         492984 non-null  object 
 1   averageRating  492984 non-null  float64
 2   numVotes       492984 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB
